In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [ ]:
df_fold = pd.read_csv('/kaggle/input/folding-30days/folds_train.csv.csv')
test_df = pd.read_csv('/kaggle/input/30-days-of-ml/test.csv')
sub_df = pd.read_csv('/kaggle/input/30-days-of-ml/sample_submission.csv')

In [ ]:
Q1 = df_fold.quantile(0.25)
Q3 = df_fold.quantile(0.75)

IQR = Q3-Q1
print(IQR)

df_fold = df_fold[~((df_fold < (Q1 -1.5 * IQR)) | (df_fold > (Q3 + 1.5 *IQR))).any(axis=1)]
df_fold.shape

In [ ]:
# Q1 = test_df.quantile(0.25)
# Q3 = test_df.quantile(0.75)

# IQR = Q3-Q1
# print(IQR)

# test_df = test_df[~((test_df < (Q1 -1.5 * IQR)) | (test_df > (Q3 + 1.5 *IQR))).any(axis=1)]
# test_df.shape

In [ ]:
features = [col for col in df_fold.columns if col not in ('id', 'target', 'kfold')]
objects_col = [col for col in features if 'cat' in col]

test_df = test_df[features]

In [ ]:
preds = []


for i in range(5):
    X_train = df_fold[df_fold['kfold'] != i].reset_index(drop=False)
    X_valid = df_fold[df_fold['kfold'] == i].reset_index(drop=False)
    X_test = test_df.copy()
    
    y_train  = X_train['target']
    y_valid  = X_valid['target']
    
    X_train = X_train[features]
    X_valid = X_valid[features]
    
    ord_encoder =  OrdinalEncoder()
    X_train[objects_col] = ord_encoder.fit_transform(X_train[objects_col])
    X_valid[objects_col] = ord_encoder.transform(X_valid[objects_col])
    X_test[objects_col] = ord_encoder.transform(X_test[objects_col])
    
    xgb = XGBRegressor( random_state=i, tree_method='gpu_hist', gpu_id=0, predictor='gpu_predictor')
    xgb.fit(X_train, y_train)
    preds_valid = xgb.predict(X_valid)
    preds_test = xgb.predict(X_test)
    preds.append(preds_test)
    print(i, mean_squared_error(y_valid, preds_valid, squared=False))

In [ ]:
predict = np.mean(np.column_stack(preds), axis=1)
print(predict)

In [ ]:
sub_df.target = predict
sub_df.to_csv("submission.csv", index=False)